<a href="https://colab.research.google.com/github/alphoenixbiz/AlphaCode-EXPLORE/blob/master/Submissions/Classification%20Model/Alfred_predict03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import datetime
import seaborn as sns
import pickle
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
%matplotlib inline
from sklearn.utils import resample
import re
import nltk
from nltk import TreebankWordTokenizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error, r2_score

### Install SQL Alchemy

In [0]:
!pip install sqlalchemy
!pip install pymssql

### Import packages

In [0]:
import sqlalchemy as sa
import pandas as pd
from sqlalchemy import create_engine

### Create a connection to our database

In [0]:
Username = 'admin'
Password = 'amazing_people'
Host = 'alphacode-explore.ccwgqdqrrmvt.eu-west-1.rds.amazonaws.com'
Port = '1433'
Database = 'Alfred'

### Create connection

In [0]:
connection = create_engine(
    'mssql+pymssql://' +
    Username + ':' + Password + '@' + Host + ':' + Port + '/' + Database
)

In [0]:
# from google.colab import drive 
# drive.mount('/content/gdrive')

In [0]:
# BankStatements = pd.read_csv("gdrive/My Drive/Colab Notebooks/DATA/project_data/BankStatements.csv")

In [0]:
# BankStatements.head()

In [0]:
# BankStatements.drop(['Unnamed: 6','Unnamed: 7','Unnamed: 8'],inplace=True,axis=1)
# BankStatements.head()

In [0]:
# BankStatements.to_sql('BankStatements', con=connection, if_exists='append', index=False,method='multi',chunksize=500)


### What is the sql query we want to perform?

In [0]:
sql_query1 = "USE Alfred Select * from BankStatements"

 ### Using pandas we now take our connection and our query to create a dataframe

In [0]:
#import the training data
train = pd.read_sql_query(sql_query1, connection)
train.head()

,Transaction Date,Posting Date,Description,Debits,Credits,Category
0,2020/02/10,2020/02/10,MTN DATA BUNDLES,499,None,COMMUNICATION
1,2020/02/08,2020/02/09,APPLE.COM/BILL CORK IE,74.99,None,ENTERTAINMENT & EAT OUT
2,2020/02/07,2020/02/09,PARKHURST HARDWARE RE PARKHURST ZA,75,None,HOME
3,2020/02/07,2020/02/08,SEATTLE RENNIES HOUSE JOHANNESBURG ZA,31.5,None,ENTERTAINMENT & EAT OUT
4,2020/02/07,2020/02/08,FLM RENNIE HOUSE EATER BRAAMFONTEIN ZA,29.98,None,ENTERTAINMENT & EAT OUT


In [0]:
train.tail()

,Transaction Date,Posting Date,Description,Debits,Credits,Category
938,None,2020-02-02,IBANK PAYMENT,200,None,GIFTS AND DONATIONS
939,None,2020-02-02,POS PURCHASE BP Filling Sta,81.6,None,TRANSPORT
940,None,2020-02-02,POS PURCHASE BP Filling Sta,28,None,TRANSPORT
941,None,2020-02-01,ACB DEBIT:EXTERNAL VIRGIN ACT,297.5,None,LIFE AND HEALTH
942,None,2020-02-01,MANAGEMENT FEE,29,None,FEES


In [0]:
# #import the test data
# test = pd.read_sql_query(sql_query2, connection)
# test.head()

# EDA

In [0]:
# get the target variable
# create our target variables
y=train.Category.apply(lambda x: x.lower())
y.head()


0              communication
1    entertainment & eat out
2                       home
3    entertainment & eat out
4    entertainment & eat out
Name: Category, dtype: object

In [0]:
y.value_counts()

transport                  295
fees                       216
entertainment & eat out    174
communication               68
food                        42
home                        32
other                       28
short-term credit           27
life and health             19
saving and investment       17
gifts & donations           15
education                    7
gifts and donations          1
interest                     1
income                       1
Name: Category, dtype: int64

In [0]:
y_unique = y.value_counts()
y_unique = pd.DataFrame(y_unique)
y_unique.head()

,Category
transport,295
fees,216
entertainment & eat out,174
communication,68
food,42


In [0]:
y_unique = y_unique.reset_index(drop=False)
y_unique.head()

,index,Category
0,transport,295
1,fees,216
2,entertainment & eat out,174
3,communication,68
4,food,42


In [0]:
y_unique.drop(['Category'], axis=1, inplace=True)  
y_unique['New_ID'] = y_unique.index
y_unique.head()

,index,New_ID
0,transport,0
1,fees,1
2,entertainment & eat out,2
3,communication,3
4,food,4


In [0]:
len(y_unique)

15

In [0]:
y =  pd.DataFrame(y)
y.head()

,Category
0,communication
1,entertainment & eat out
2,home
3,entertainment & eat out
4,entertainment & eat out


In [0]:
y.Category.values

array(['communication', 'entertainment & eat out', 'home',
       'entertainment & eat out', 'entertainment & eat out',
       'entertainment & eat out', 'entertainment & eat out',
       'entertainment & eat out', 'entertainment & eat out',
       'entertainment & eat out', 'saving and investment',
       'entertainment & eat out', 'entertainment & eat out',
       'entertainment & eat out', 'saving and investment',
       'entertainment & eat out', 'entertainment & eat out',
       'entertainment & eat out', 'entertainment & eat out', 'other',
       'saving and investment', 'life and health', 'other',
       'entertainment & eat out', 'entertainment & eat out',
       'entertainment & eat out', 'entertainment & eat out',
       'life and health', 'saving and investment',
       'saving and investment', 'communication', 'life and health',
       'short-term credit', 'entertainment & eat out',
       'entertainment & eat out', 'entertainment & eat out',
       'entertainment & eat out

In [0]:
mapping = dict(y_unique[['index', 'New_ID']].values)
y['Item_ID'] = y.Category.map(mapping)
y.head()

,Category,Item_ID
0,communication,3
1,entertainment & eat out,2
2,home,5
3,entertainment & eat out,2
4,entertainment & eat out,2


In [0]:
y.drop(['Category'], axis=1, inplace=True)
y.head()

,Item_ID
0,3
1,2
2,5
3,2
4,2


In [0]:
len(y)

943

In [0]:
features = train.Description.values
features2 = train.Description.values

In [0]:
features[:5]

array(['MTN DATA BUNDLES', 'APPLE.COM/BILL CORK IE',
       'PARKHURST HARDWARE    RE PARKHURST ZA',
       'SEATTLE RENNIES HOUSE JOHANNESBURG ZA',
       'FLM RENNIE HOUSE EATER BRAAMFONTEIN ZA'], dtype=object)

# Text to Features

In [0]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

lem = WordNetLemmatizer()
# ps = PorterStemmer()
corpus =[]
# cleaning text
for i in range(0,len(features)):

    item = re.sub('[^a-zA-Z]', ' ',str(features[i])) 
    item = item.lower()
    item = item.split()
    stop_wrds = set(stopwords.words('english'))
    item = [lem.lemmatize(word, "v") for word in item if not word in stop_wrds]
    item = ' '.join(item)
    corpus.append(item)

new_features = corpus

In [0]:
new_features[:5]

['mtn data bundle',
 'apple com bill cork ie',
 'parkhurst hardware parkhurst za',
 'seattle rennies house johannesburg za',
 'flm rennie house eater braamfontein za']

In [0]:
# txt = np.array(['acb debit external virgin act'])


In [0]:
def tokenize(text):
    return([text.split(' ', 1)[0].strip()])
# Create Ranked Statistical Features with TfidfVectorizer, best choice because insignificant words recieve a lower rank
from sklearn.feature_extraction.text import CountVectorizer
obj = TfidfVectorizer()
# obj = CountVectorizer(decode_error = 'ignore', stop_words=stop_wrds,analyzer='char',max_features= 10,ngram_range=(1,10))
obj = CountVectorizer(decode_error = 'ignore',analyzer='char',ngram_range=(1,10))

final_features = obj.fit_transform(new_features)
# print(final_features[0])
final_features.shape

(943, 21216)

In [0]:
final_features

<943x21216 sparse matrix of type '<class 'numpy.int64'>'
	with 112439 stored elements in Compressed Sparse Row format>

In [0]:
# import joblib
# # save the model to disk
# filename = 'fCountVectorizer.pkl'
# joblib.dump(obj, filename)

In [0]:
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# # 1. Authenticate and create the PyDrive client.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)  

# # get the folder id where you want to save your file
# file = drive.CreateFile()
# file.SetContentFile('fCountVectorizer.pkl')
# file.Upload() 

In [0]:
# CountVectorizer = joblib.load('CountVectorizer.pkl')
# obj = CountVectorizer()
# final_features = obj.fit_transform(new_features)
# # print(final_features[0])
# final_features

In [0]:

# split the train set to create a validation set
X_train, X_test, y_train, y_test = train_test_split(final_features, y.values.ravel(), test_size=0.20)


In [0]:
X_train

<754x21216 sparse matrix of type '<class 'numpy.int64'>'
	with 90060 stored elements in Compressed Sparse Row format>

In [0]:
#import model
import xgboost as xgb
from xgboost import XGBClassifier
cls = XGBClassifier(n_estimators=100)

In [0]:
#import model
# from sklearn.neighbors import KNeighborsClassifier
# cls = KNeighborsClassifier()

In [0]:
# train initial model
cls = cls.fit(X_train, y_train.ravel())

In [0]:
# import joblib
# # save the model to disk
# filename = 'fmodel.pkl'
# joblib.dump(cls, filename)

In [0]:


# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# # 1. Authenticate and create the PyDrive client.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)  

# # get the folder id where you want to save your file
# file = drive.CreateFile()
# file.SetContentFile('fmodel.pkl')
# file.Upload() 


In [0]:
# from google.colab import files
# files.download('model.sav')

In [0]:
# # load the model from disk
# loaded_model = joblib.load('model.sav')
# result = loaded_model.predict(X_test)
# print(result[0:5])

In [0]:
# final_features

In [0]:
# Making the predictions
pred1 = cls.predict(X_test)

In [0]:
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, pred1))

Mean squared error: 2.14


In [0]:
# performance matrics and model eveluation using sklearn.metrics inbuilt classification metric
print(classification_report(y_test, pred1))

              precision    recall  f1-score   support

           0       0.94      0.92      0.93        49
           1       0.94      1.00      0.97        49
           2       0.81      0.87      0.84        39
           3       1.00      1.00      1.00        15
           4       0.71      0.50      0.59        10
           5       0.67      0.80      0.73         5
           6       1.00      0.25      0.40         4
           7       0.75      0.86      0.80         7
           8       0.33      0.50      0.40         4
           9       0.00      0.00      0.00         2
          10       1.00      1.00      1.00         1
          11       1.00      0.50      0.67         4

    accuracy                           0.87       189
   macro avg       0.76      0.68      0.69       189
weighted avg       0.87      0.87      0.86       189



In [0]:
y_predicted = pd.DataFrame({'PredictItem_ID':pred1})
y_predicted.head()

,PredictItem_ID
0,2
1,2
2,10
3,2
4,2


In [0]:
mapping = dict(y_unique[['New_ID','index']].values)
y_predicted['ItemCategory_New'] = y_predicted.PredictItem_ID.map(mapping)
y_predicted.head()

,PredictItem_ID,ItemCategory_New
0,2,entertainment & eat out
1,2,entertainment & eat out
2,10,gifts & donations
3,2,entertainment & eat out
4,2,entertainment & eat out


In [0]:
# import joblib
# # save the model to disk
# filename = 'fmapping.pkl'
# joblib.dump(mapping, filename)



# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# # 1. Authenticate and create the PyDrive client.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)  

# # get the folder id where you want to save your file
# file = drive.CreateFile()
# file.SetContentFile('fmapping.pkl')
# file.Upload() 

In [0]:

y_predicted = np.array(y_predicted.drop('PredictItem_ID', axis=1))
y_predicted[:5]

array([['entertainment & eat out'],
       ['entertainment & eat out'],
       ['gifts & donations'],
       ['entertainment & eat out'],
       ['entertainment & eat out']], dtype=object)

# submission

In [0]:
# save predictions
submission = pd.DataFrame({'description' : features2[X_train.shape[0]:],'category' : y_predicted.ravel()}) 

In [0]:
submission.category.value_counts()

fees                       52
transport                  48
entertainment & eat out    42
communication              15
short-term credit           8
food                        7
home                        6
life and health             6
education                   2
other                       1
saving and investment       1
gifts & donations           1
Name: category, dtype: int64

In [0]:
submission.head()

,description,category
0,FEE - PRE-PAID TOP UP,entertainment & eat out
1,CHECKERS BRAM 5326*9249 11 JAN,entertainment & eat out
2,C*ENGEN CONRA 5326*9249 13 JAN,gifts & donations
3,C*ENGEN PAARD 5326*9249 13 JAN,entertainment & eat out
4,I*MR D FOOD Z 5326*9249 14 JAN,entertainment & eat out


In [0]:
# submission.to_sql('NLP_model_results', con=connection, if_exists='append', index=False,method='multi',chunksize=500)
